# Hotdog or NotHotdog?
## Using PowerAI to Deploy an iOS Model for CoreML

<strong><a href=https://developer.ibm.com/indexconf/sessions/#!?id=5466>Index 2018 </a></strong></br>

Yes, really. <a href="https://twitter.com/mccoyjus">Justin McCoy</a> and <a href="https://twitter.com/dokun24">David Okun</a> will walk you through how to use PowerAI in the Cloud with Nimbix, to quickly train and deploy a Model that can tell you whether or not your lunchtime nutritional choice is the right one - all with the camera of the mobile phone in your pocket. All you need are some photos, descriptions of them, and you can be up and running with a model to stream video through in no time flat.

So you want to identify hotdogs, great, summer is just around the corner, and you can never be too careful of what you're eating. 

During this demo we will walk you through the steps, and technologies necessary to train a Deep Learning model using a Convolutional Neural Network, saving it into a format that can be loaded on an iOS device.

I'm sure you've seen the eposide of Silicon Valley, 



"edge computing"

## Agenda
1. Setup Environment 
2. Acquire Data
3. Build Convolutional Neural Network
4. Train Convolutional Neural Network
5. Evaluate Model
6. Make predictions
7. Take a closer look at generated filters
8. Save to CoreML
9. Transfering Learning

# 1. Setup Environment

### Install and import libraries

In [ ]:
# Late in this Notebook the Keras Model is converted to Apple's coreML format; only Keras 2.0.4
#  is supported as of creation of this notebook
!sudo pip install keras==2.0.4
!sudo pip install coremltools

In [ ]:
from __future__ import division

import json
from matplotlib import pyplot
import math
import numpy as np
import os
import sys
import time

import keras
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import BatchNormalization

from keras.models import Model
from keras.models import model_from_json
from keras.models import Sequential
from keras.models import load_model

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator 

from keras import callbacks
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

from keras import backend as K


from tensorflow import Tensor

from keras.engine import InputLayer
import coremltools

%matplotlib inline

### Setup global variables

In [ ]:
ITERATION = 7

DATA_PATH = 'hotdog_data'
TRAINING_DATA_PATH = DATA_PATH + '/training_set'
VALIDATION_DATA_PATH = DATA_PATH + '/validation_set'
TEST_DATA_PATH = DATA_PATH + '/test_set'

OUTPUT_PATH = 'output_' + time.strftime("%d-%m-%Y_") + str(ITERATION)
MODEL_JSON_PATH = OUTPUT_PATH + '/seefood_model.json'
MODEL_WEIGHTS_PATH = OUTPUT_PATH + '/seefood_model_weights.h5'
MODEL_EPOCH_PATH = OUTPUT_PATH + '/seefood_weights-{epoch:02d}-{val_acc}.hdf5'

COREML_MODEL_PATH = OUTPUT_PATH + '/seefood_model.mlmodel'
COREML_META_DESCRIPTION = 'SeeFood: Model to classify images as either hotdog or nothotdog'
COREML_META_AUTHOR = 'Justin A. McCoy'
COREML_META_INPUT_DESCRIPTION = 'Image of food that might be a hotdog'


# input image dimensions
IMG_ROWS, IMG_COLS = 128, 128
IMG_CHANNELS = 3

# Number of images gererated at each invocation of the ImageDataGenerator
# Batchsize is an important value when training a CNN. too large a number can lead to out of memory errors, 
#  and lower accuracy. https://arxiv.org/abs/1606.02228 
# When you have a high batch size in compairison to the number of training samples you make bigger jumps during
# graident descent, this can lead you to the minimuma faster but it's possib
BATCH_SIZE = 16

EPOCHS = 25

HOTDOG = 0
NOTHOTDOG = 1

CLASSIFICATION = ['HOTDOG', 'NOTHOTDOG']

### Create output directory

In [ ]:
!mkdir "$OUTPUT_PATH"
print('Created path: {}'.format(OUTPUT_PATH))

### Common methods used throughout notebook

In [ ]:
def show_images(images, cols=1, title=""):
    """Display a list of images in a single plot with matplotlib.
    
    Parameters
    ---------
    images: List of np.arrays compatible with plt.imshow.
    
    cols (Default = 1): Number of columns in figure (number of rows is 
                        set to np.ceil(n_images/float(cols))).
    """
    n_images = len(images)
    fig = pyplot.figure()
    fig.suptitle(title, fontsize=16)
    for n, image in enumerate(images):
        pyplot.subplot(cols, np.ceil(n_images/float(cols)), n + 1)
        pyplot.imshow(image)
        pyplot.axis('off')
    pyplot.show()
    
    
def get_best_model():
    return load_model(MODEL_WEIGHTS_PATH)

# 2. Acquire Data

### Aquire and classify training data for Hotdogs

There are several places to aquire datasets that are classififed. for this example I've used Image.net to aquire images of hotdogs and food items that are not hotdogs

www.image-net.org/

ImageNet is a database of images organized according to the WordNet hierarchy, where each concept described by multiple words is grouped in a 'synonym set'. The synsets are thumbnails of images and contain links to images that have been classified.

A major drawback of ImageNet revolves around where images are located; ImageNet doesn't keep a database of images but links to where images are, or were at one time. ImageNet doesn't save you from the task of cleaning and reviewing your data before jumping into the fun stuff.

#### Downloading images


In [ ]:
!wget https://ibm.box.com/s/ig5ao996ew9mqutfbgs0o2otophuwe1h
tar -xzvf hotdog_data.tar.gz

### Look at the training and validation data
It's important to understand your data balance between classes.

In [ ]:
# How many images do we have to work with in our training and validation sets?

hd_training_set_size = !ls -alR '$TRAINING_DATA_PATH'/hotdog | wc -l
hd_training_set_size = hd_training_set_size[0]

nhd_training_set_size = !ls -alR '$TRAINING_DATA_PATH'/not-hotdog | wc -l
nhd_training_set_size = nhd_training_set_size[0]

validation_set_size = !ls -alR '$VALIDATION_DATA_PATH'/hotdog | wc -l
validation_set_size = validation_set_size[0]

test_set_size = !ls -alR '$TEST_DATA_PATH'/ | wc -l
test_set_size = test_set_size[0]

print("Hotdog training examples:\t{}".format(hd_training_set_size))
print("NotHotdog training examples:\t{}".format(nhd_training_set_size))
print("Hotdog validation examples:\t{}".format(validation_set_size))
print("Test examples:\t{}".format(test_set_size))

### Data Augmentation with Generators
Using Keras's ImageDataGenerator we solve several problems with large datasets

1. Eliminate the need to store our entire dataset in memory
2. Supplement dataset with additional training images
3. Improve model robustness by producing different color and oritntation profiles of images
4. Mean-Normalization - Image pixel values are usually integers between the range of 1 and 255. Using such large numbers in models can cause overflow. To fix this the ImageDataGenerator provides a rescale function to scale the original pixel values by a sacaling factor, in our case this is 255. Leaving values between 0 and 1 for each pixel value.

ere we’ve rescaled the image data so that each pixel lies in the interval [0, 1] instead of [0, 255]. It is always a good idea to normalize the input so that each dimension has approximately the same scale.

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
    rotation_range=40,
#    width_shift_range=0.2,
#    height_shift_range=0.2,
#    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale = 1./255)

print("training_set")
training_set = train_datagen.flow_from_directory(TRAINING_DATA_PATH,
    target_size = (IMG_ROWS, IMG_COLS),
    batch_size = BATCH_SIZE,
    class_mode = 'binary')
    
print("validation_set")
validation_set = validation_datagen.flow_from_directory(VALIDATION_DATA_PATH,
    target_size = (IMG_ROWS, IMG_COLS),
    batch_size = BATCH_SIZE,
    class_mode = 'binary')

print("test_set")
test_datagen = ImageDataGenerator()
test_set = test_datagen.flow_from_directory(TEST_DATA_PATH,
                                            target_size=(IMG_ROWS, IMG_COLS),
                                            batch_size=488,
                                            class_mode='binary')

In [ ]:
# Using Keras's ImageDataGenerator we generate new images from transformations of our initial training set
TRAINING_IMAGES = 20000
VALIDATION_IMAGES = validation_set.samples # No augmentation of the validation images, just use what's there

### Look at subset of sample generated images in the training_set

In [ ]:
from matplotlib import pyplot
%matplotlib inline
    
x,y = training_set.next()  # Generates BATCH_SIZE images on every invocation

# Only interested in a subset of the images
x = x[:9]
y = y[:9]

fig = pyplot.figure()
title = fig.suptitle('training_set examples', fontsize=16)
title.set_position([.5, 1.05])
for n, (img, title) in enumerate(zip(x,y)):
    a = fig.add_subplot(3, 3, n+1)
    pyplot.imshow(img)
    a.set_title(title)
    pyplot.axis('off')
#fig.set_dpi(300)
pyplot.show()

print(training_set.class_indices)

# 3. Build Convolutional Neural Network

### A Convolutional Neural Network (CNN) 

Each filter in a CNN, learns different characteristic of an image.

Keras allows us to specify the number of **filters** we want and the size of the filters. So, in our first layer, 32 is number of filters and (3, 3) is the size of the filter. We also need to specify the shape of the input which is (28, 28, 1), but we have to specify it only once.

The second layer is the **Activation layer**. We have used ReLU (rectified linear unit) as our activation function. ReLU function is f(x) = max(0, x), where x is the input. It sets all negative values in the matrix ‘x’ to 0 and keeps all the other values constant. It is the most used activation function since it reduces training time and prevents the problem of vanishing gradients.

The third layer is the **MaxPooling layer**. MaxPooling layer is used to down-sample the input to enable the model to make assumptions about the features so as to reduce over-fitting. It also reduces the number of parameters to learn, reducing the training time.

After creating all the **convolutional layers**, we need to flatten them, so that they can act as an input to the Dense layers.

**Dense layers** are keras’s alias for Fully connected layers. These layers give the ability to classify the features learned by the CNN.

**Dropout** is the method used to reduce overfitting. It forces the model to learn multiple independent representations of the same data by randomly disabling neurons in the learning phase. In our model, dropout will randomnly disable 20% of the neurons.

The second last layer is the Dense layer with 1 neuron. The neurons in this layer should be equal to the number of classes we want to predict as this is the output layer.




In [ ]:
K.set_learning_phase(1) # https://github.com/keras-team/keras/issues/2310
classifier = Sequential()

In [ ]:
classifier.add(Conv2D(64, (3, 3), input_shape = (IMG_ROWS, IMG_COLS, IMG_CHANNELS), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(Dropout(0.4))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(128, (3, 3), activation='relu'))
classifier.add(Dropout(0.4))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Flatten())
classifier.add(Dense(64))
classifier.add(Activation('relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(1))
classifier.add(Activation('sigmoid'))


The cross-entropy loss calculates the error rate between the predicted value and the original value. The formula for calculating cross-entropy loss is given here. Because we have two classes we used binary_crossentropy.

The Adam optimizer is an improvement over SGD(Stochastic Gradient Descent). The optimizer is responsible for updating the weights of the neurons via backpropagation. It calculates the derivative of the loss function with respect to each weight and subtracts it from the weight. This is how a neural network learns.


In [ ]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Save CNN as JSON


In [ ]:
#serialize model to JSON
classifier_json = classifier.to_json()
with open(MODEL_JSON_PATH, "w") as json_file:
    json_file.write(classifier_json)
    

# 4. Train Convolutional Neural Network

### Metrics to monitor
When training your convolutional neural network you're monitoring two things at each epoch for both the training and validation stages: **accuracy**, and **loss**.  As you monitor these metrics hopefully the accurracy goes up and the loss goes down. 

When training it's possible to **underfit** and **overfit**. Underfitting occurs when the accuracy on the training set is lower then the accuracy on the validation set; a poor performing model. Overfitting occurs when the training loss contines to go down, but the validation loss continues to rise.

Finding the sweet spot between underfitting and overfitting is crucial to developing a model that generalizes well with unseen data.

**Dropout Regularization** is one method that's used to reduce overfitting. Dropout randomly turns off neurons during training, reducing their weight further down the network. This encourages the network to find additional neurons building separate representations of the class within the network. Increasing the dropout rate between layers combats against overfitting; a good starting point is between 0.20 and 0.50.  

#### *Finding the right balance is more of an art then science, and takes some time to experiment.*

![XKCD.com/1838](https://imgs.xkcd.com/comics/machine_learning.png)


### Setup callbacks to monitor metrics

**Callbacks** are functions applied at given stages of training. Below is an example demostrating how to create a new callback, reporting the *validation loss* and *validation accuracy* at the end of each epoch.

Keras comes with many callback functions, including ModelCheckpoint used to save the weights after every epoch as seen below.

In [ ]:
class LossHistory(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_epoch_end(self, epoch, logs={}):
        self.losses.append(logs.get('val_loss'))
        print('-----------------------------------------------------------------------')
        print('Epoch ' + str(epoch) + ' - Validation loss: ' + str(logs.get('val_loss')) + ' accuracy : ' + str(logs.get('val_acc')))
        print('-----------------------------------------------------------------------')


In [ ]:
checkpoint_all = ModelCheckpoint(filepath= MODEL_EPOCH_PATH, verbose=1, save_best_only=False)
checkpoint_best = ModelCheckpoint(filepath= MODEL_WEIGHTS_PATH, verbose=1, save_best_only=True)
loss_history = LossHistory()

In [ ]:
history = classifier.fit_generator(training_set,
    steps_per_epoch = TRAINING_IMAGES//BATCH_SIZE,
    epochs = 50,
    validation_data = validation_set,
    validation_steps = VALIDATION_IMAGES//BATCH_SIZE,
    callbacks = [loss_history, checkpoint_all, checkpoint_best])

**Hyperparameter optimization is necessary to improve the models accuracy**

# 5. Evaluate Model

In [ ]:
classifier.summary()

### Plot Accuracy and Loss over Epochs
Plot the validation accuracy vs training accuracy, and the validation_loss vs training_loss

In [ ]:

pyplot.plot(history.epoch,history.history['val_acc'],label='validation accuracy')
pyplot.plot(history.epoch,history.history['acc'],label='training accuracy')


pyplot.legend(loc=0)
pyplot.xlabel('epoch')
pyplot.ylabel('accuracy')
pyplot.grid(True)
pyplot.show()

In [ ]:
pyplot.plot(history.epoch,history.history['val_loss'],label='validation loss')
pyplot.plot(history.epoch,history.history['loss'],label='training loss')


pyplot.legend(loc=0)
pyplot.xlabel('epoch')
pyplot.ylabel('loss')
pyplot.grid(True)
pyplot.show()

### Load the best weights from the training 

In [ ]:
model = get_best_model()

In [ ]:
# The evaluate_generator method returns a tuple including the loss and accuracy of a model
results = model.evaluate_generator(validation_set, 1)
print("The model has a {}% accuracy, with a loss of {}.".format(results[1]*100, results[0])) 

### Create a Confusion Matrix on with test data

It's important to test your model with data that hasn't been used during training for validation. 

|               |   **HOTDOG**   |  **NOTHOTDOG** |
| ------------- | -------------- | -------------- |
| **HOTDOG**    | True Positive  | False Positive |
| **NOTHOTDOG** | False Negative | True Negative  |

In [ ]:
# Creating the Confusion Matrix on our test data

X_test, y_test = test_set.next()
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

# 6. Make Predictions

In [ ]:
#Load the best weights from the training 
model = get_best_model()

In [ ]:
def make_prediction(model, img):
    """Display a list of images in a single plot with matplotlib.
    
    Parameters
    ---------
    images: List of np.arrays compatible with plt.imshow.
    
    cols (Default = 1): Number of columns in figure (number of rows is 
                        set to np.ceil(n_images/float(cols))).
    """
    test_image = image.img_to_array(img)
    test_image = np.expand_dims(test_image, axis = 0)
    return model.predict(test_image, verbose=0)

In [ ]:
false_positives=[]
for file in os.listdir(TEST_DATA_PATH + '/nothotdog'):
    img = image.load_img(TEST_DATA_PATH + '/nothotdog/' + file, target_size = (IMG_COLS, IMG_COLS))
    if make_prediction(model, img) == HOTDOG:
        false_positives.append(img)     
 
show_images(false_positives, 3, title="False Positives")



In [ ]:
false_negatives=[]
for file in os.listdir(TEST_DATA_PATH + '/hotdog'):
    img = image.load_img(TEST_DATA_PATH + '/hotdog/' + file, target_size = (IMG_COLS, IMG_COLS))
    if make_prediction(model, img) == NOTHOTDOG:
        false_negatives.append(img)
        

show_images(false_negatives, 10, title="False Negatives")

### Download images from the web to evaluate model

In [ ]:
!wget https://www.dietsinreview.com/diet_column/wp-content/uploads/2010/07/joey-chestnut-nathans-famous-hot-dog-eating-contest.jpg -O hotdog.jpg 

In [ ]:
test_image = image.load_img('hotdog.jpg', target_size = (IMG_ROWS, IMG_COLS))
pyplot.imshow(test_image)
pyplot.show()

test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result = classifier.predict(test_image, verbose=1)

print(CLASSIFICATION[int(result[0][0])])


In [ ]:
!wget https://img.webmd.com/dtmcms/live/webmd/consumer_assets/site_images/dam/editorial/childrens-health/miscellaneous/how-to-change-teen-eating-habits/graphics/thumbnails/final/how-to-change-teen-eating-habits-375x321.jpg -O nothotdog.jpg

In [ ]:
test_image = image.load_img('nothotdog.jpg', target_size = (IMG_ROWS, IMG_COLS))
pyplot.imshow(test_image)
pyplot.show()

test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result = classifier.predict(test_image, verbose=1)

print(CLASSIFICATION[int(result[0][0])])

# 7. Take a closer look at generated filters
The CNN has several layers, including various filtering layers to identify important features of the image for the classification task. Let's look at the filters to see what areas of an image are identified as important for the hotdog, no hotdog classificatio
Inspired by code here: https://github.com/mingruimingrui/Convolution-neural-networks-made-easy-with-keras

In [ ]:
model = get_best_model()

In [ ]:
def get_layer_dict(model):
    return dict([(layer.name, layer) for layer in model.layers if (layer.name.find('dense') > -1) | (layer.name.find('conv') > -1)])

In [ ]:
layers = get_layer_dict(model)
layers

In [ ]:
def deprocess_image(img):
    # normalize tensor: center on 0., ensure std is 0.1
    img -= img.mean()
    img /= (img.std() + 1e-5)
    img *= 0.1

    # clip to [0, 1]
    img += 0.5
    img = np.clip(img, 0, 1)

    # convert to RGB array
    img *= 255
    img = np.clip(img, 0, 255).astype('uint8')
    return img


In [ ]:
def plot_hidden_filter_layers(model, layer, num_plot=16):
    _ = pyplot.suptitle(layer.name)

    # we shall only plot out 16(default) as there are too many filters to visualize

    layer_output = layer.output
    output_shape = layer.output_shape
    sub_plot_height = math.ceil(np.sqrt(num_plot))
    nb_filters = output_shape[len(output_shape) - 1]

    # here we need to conduct gradient acdent on each filter
    counter = 0
    for i in range(nb_filters):
        if counter < num_plot:
            # conv layers have different outputs than dense layers therefore different loss function sizes
            if layer.name.find('conv') != -1:
                loss = K.mean(layer_output[:,:,:,np.random.randint(nb_filters)])
            else:
                loss = K.mean(layer_output[:,np.random.randint(nb_filters)])

            # randomise initial input_img and calc gradient
            input_img = model.input#np.expand_dims(np.ones(X_shape[1:]), axis=0)
            grads = K.gradients(loss, input_img)[0]

            # normalize gradient
            grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

            # this function returns the loss and grads given the input picture
            iterate = K.function([input_img], [loss, grads])

            # we start from a gray image with some noise
            input_img_data = np.random.rand(1,IMG_ROWS, IMG_COLS, IMG_CHANNELS) * 0.1 + 0.5

            # run gradient ascdent for 20 steps
            for j in range(40):
                loss_value, grads_value = iterate([input_img_data])
                input_img_data += grads_value

            # deprocess_image and plot if found
            if loss_value > 0:
                img = deprocess_image(input_img_data[0])
                ax = pyplot.subplot(sub_plot_height, sub_plot_height, counter+1)
                _ = pyplot.axis('off')
                _ = ax.set_xticklabels([])
                _ = ax.set_yticklabels([])
                _ = ax.set_aspect('equal')
                _ = pyplot.imshow(img.squeeze(), cmap='inferno')

                counter += 1

    _ = pyplot.show()

In [ ]:
for layer_name in layers:
    plot_hidden_filter_layers(model, layers[layer_name])

### Visualize image through each filter layer
Run an image through each convolutional layer and display the different filters applied to the image. Each filter focuses on features within the image. This gives you a better understanding of what the CNN is looking for when making it's final decision; hotdog or nothotdog.

In [ ]:
model = get_best_model()

In [ ]:
def get_image_from_generator():
    '''Generate some validation data and select a single sample image'''
    X, y = validation_set.next()
    img = X[0].reshape(-1,IMG_ROWS,IMG_COLS,IMG_CHANNELS)
    return img

In [ ]:
def get_conv_intermediate_layers(my_model):
    '''Returns the names of each convolutional layer in the model'''
    conv_layers = []
    for layer in my_model.layers:
        if 'conv' in layer.name:
            # Build a new model with the input from the original 
            # model, but with the output of specific layer.
            conv_layers.append(Model(inputs=my_model.input,
                                     outputs=my_model.get_layer(layer.name).output))            
    return conv_layers

In [ ]:
def plot_hidden_layers(my_model, img):
    to_visual = my_model.predict(img)
    to_visual = to_visual.reshape(to_visual.shape[1:])
    _ = pyplot.figure()

    sub_plot_height = math.ceil(np.sqrt(to_visual.shape[2]))
    for i in range(to_visual.shape[2]):
        ax = pyplot.subplot(sub_plot_height, sub_plot_height, i+1)
        _ = pyplot.axis('off')
        _ = ax.set_xticklabels([])
        _ = ax.set_yticklabels([])
        _ = ax.set_aspect('equal')
        _ = pyplot.imshow(to_visual[:, :, i], cmap='inferno')
        

In [ ]:
conv_models = get_conv_intermediate_layers(model)
img = get_image_from_generator()

_ = pyplot.imshow(img.reshape(img.shape[1:]))
_ = pyplot.title('Hotdog')

index = 0
for my_model in conv_models:
    index += 1
    
    plot_hidden_layers(my_model, img)

pyplot.show()

# 8. Convert Keras Model to Apple's coreML

#### Load the existing Keras model from disk

In [ ]:
# Load the best weights from the training 
model = get_best_model()

#### Set model properties and save as coreML

In [ ]:
output_labels = ['nothotdog']
coreml_model = coremltools.converters.keras.convert(MODEL_WEIGHTS_PATH, input_names='image',image_input_names = 'image',class_labels = output_labels)   
coreml_model.author = AUTHOR   
coreml_model.short_description = COREML_META_DESCRIPTION 
coreml_model.input_description['image'] = COREML_META_INPUT_DESCRIPTION


coreml_model.save(COREML_MODEL_PATH)

print coreml_model   

# 9. Transfer Learning


With the release of MobileNet https://research.googleblog.com/2017/06/mobilenets-open-source-models-for.html it's now possible to get much more accurate image classification on the edge of a network such as your smartphone.

# TODO: Import MobileNet and build classifier from there

In [ ]:
!wget http://download.tensorflow.org/models/mobilenet_v1_1.0_224_2017_06_14.tar.gz
!tar -xzvf mobilenet_v1_1.0_224_2017_06_14.tar.gz
!ls

In [ ]:
from keras.applications.mobilenet import MobileNet

In [ ]:
from keras import applications

# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

bottleneck_features_validation = model.predict_generator(
        validation_set, VALIDATION_IMAGES // BATCH_SIZE)

np.save(open('bottleneck_features_validation.npy', 'w'), bottleneck_features_validation)
